In [1]:
%pip install tensorflow tf2onnx onnxruntime numpy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<5,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.3 MB 4.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/11.3 MB 2.2 MB/s eta 0:00:05
   ----- ---------------------------------- 1.6/11.3 MB 2.0 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/11.3 MB 2.3 MB/s eta 0:00:04
   -------- ------------------------------- 2.4/11.3 MB 2.0 MB/s eta 0:00:05
   ----------- ---------------------------- 3.1/11.3 MB 2.1 MB/s eta 0:00:04
   ------------ --------------------------- 3.4/11.3 MB 2.1 MB/s eta 0:00:04
   --------------- ------------------------ 4.5/11.3 MB 2.3 MB/s eta 0:00:03
   ---------------- ----------------------- 4.7/11.3 MB 2.3 MB/s eta 0:00:03
   ------------------ --------------------- 5.2/11.3 MB 2.2 MB/s eta 0:00:03
   ------------------- -------------------- 5.5/11.3 MB 2.3 MB/s eta 0:00:03
   ---------

In [2]:
import tensorflow as tf
import tf2onnx
import numpy as np
import onnx
import onnxruntime as ort
import time
import matplotlib.pyplot as plt
# Step 1: Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0 # Normalize the images
# Reshape data to add a channel dimension
x_train = x_train[..., np.newaxis].astype("float32")
x_test = x_test[..., np.newaxis].astype("float32")
# Step 2: Define a simple CNN model
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu',input_shape=(28, 28, 1)),
tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dense(10, activation='softmax')
])
# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
metrics=['accuracy'])
model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test)) #Train for 1 epoch for demo purposes
# Step 3: Convert the model to ONNX format
onnx_model_path = "mnist_cnn_model.onnx"
spec = (tf.TensorSpec((None, 28, 28, 1), tf.float32, name="input"),)
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec,output_path=onnx_model_path)
print(f"Model exported to {onnx_model_path}")
# Step 4: Load and validate the ONNX model
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)
print("ONNX model is valid")
# Step 5: Set up ONNX Runtime session
ort_session = ort.InferenceSession(onnx_model_path)
tf.config.run_functions_eagerly(True)


ImportError: DLL load failed while importing onnx_cpp2py_export: A dynamic link library (DLL) initialization routine failed.

In [8]:
# Step 8: Measure and plot inference time for TensorFlow and ONNX models
def measure_inference_time(tf_model, ort_session, x_test, num_runs=100):
    tf_times = []
    onnx_times = []
    for _ in range(num_runs):
        # Select a single random sample image and ensure correct shape for the model
        test_image = np.expand_dims(x_test[np.random.randint(len(x_test))],axis=0)
        # TensorFlow inference time
        start_time = time.time()
        tf_output = tf_model.predict(test_image)
        tf_times.append(time.time() - start_time)
        # ONNX inference time
        start_time = time.time()
        onnx_output = ort_session.run(None, {'input': test_image.astype(np.float32)})
        onnx_times.append(time.time() - start_time)
        
    return tf_times, onnx_times
    
# Run inference timing
tf_times, onnx_times = measure_inference_time(tf_model, ort_session, x_test)
# Step 9: Plot the inference time comparison
plt.figure(figsize=(10, 5))
plt.plot(tf_times, label="TensorFlow Inference Time")
plt.plot(onnx_times, label="ONNX Inference Time")
plt.xlabel("Run")
plt.ylabel("Inference Time (seconds)")
plt.legend()
plt.title("Inference Time Comparison Between TensorFlow and ONNX Runtime")
plt.show()

NameError: name 'tf_model' is not defined

In [ ]:
%pip install onnx2pytorch